In [18]:
# !where python

In [17]:
# !pip install llama-cpp-python
# !pip install langchain langchain-community sentence-transformers chromadb
# !pip install pypdf requests pydantic tqdm

In [1]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import LlamaCpp
import requests
from tqdm import tqdm
import time

In [4]:
model_path = "llama-2-7b-chat.Q4_K_M.gguf"

if not os.path.exists(model_path):
    print(f"Downloading {model_path}...")
    # You may want to replace the model URL by another of your choice
    model_url = "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf"
    response = requests.get(model_url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(model_path, 'wb') as f:
        for data in tqdm(response.iter_content(chunk_size=1024), total=total_size//1024):
            f.write(data)
    print("Download complete!")

3985356it [03:11, 20819.94it/s]                                                                                                                            

Download complete!


In [13]:
os.makedirs("docs", exist_ok=True)

# Sample text for demonstration purposes
with open("docs/sample.txt", "w") as f:
    f.write("""
    Retrieval-Augmented Generation (RAG) is a technique that combines retrieval-based and generation-based approaches
    for natural language processing tasks. It involves retrieving relevant information from a knowledge base and then 
    using that information to generate more accurate and informed responses.
    
    RAG models first retrieve documents that are relevant to a given query, then use these documents as additional context
    for language generation. This approach helps to ground the model's responses in factual information and reduces hallucinations.
    
    The llama.cpp library is a C/C++ implementation of Meta's LLaMA model, optimized for CPU usage. It allows running LLaMA models
    on consumer hardware without requiring high-end GPUs.
    
    LocalAI is a framework that enables running AI models locally without relying on cloud services. It provides APIs compatible
    with OpenAI's interfaces, allowing developers to use their own models with the same code they would use for OpenAI services.
    """)

documents = []
for file in os.listdir("docs"):
    if file.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join("docs", file))
        documents.extend(loader.load())
    elif file.endswith(".txt"):
        loader = TextLoader(os.path.join("docs", file))
        documents.extend(loader.load())

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

chunks = text_splitter.split_documents(documents)

In [6]:
print(chunks)

[Document(metadata={'source': 'docs\\sample.txt'}, page_content="Retrieval-Augmented Generation (RAG) is a technique that combines retrieval-based and generation-based approaches\n    for natural language processing tasks. It involves retrieving relevant information from a knowledge base and then \n    using that information to generate more accurate and informed responses.\n\n    RAG models first retrieve documents that are relevant to a given query, then use these documents as additional context\n    for language generation. This approach helps to ground the model's responses in factual information and reduces hallucinations.\n\n    The llama.cpp library is a C/C++ implementation of Meta's LLaMA model, optimized for CPU usage. It allows running LLaMA models\n    on consumer hardware without requiring high-end GPUs."), Document(metadata={'source': 'docs\\sample.txt'}, page_content="The llama.cpp library is a C/C++ implementation of Meta's LLaMA model, optimized for CPU usage. It allow

In [14]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

In [8]:
llm = LlamaCpp(
    model_path=model_path,
    temperature=0.7,
    max_tokens=2000,
    n_ctx=4096,
    verbose=False
)

llama_init_from_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64


In [9]:
template = """
Answer the question based on the following context:

{context}

Question: {question}
Answer:
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [10]:
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [11]:
def ask_question(question):
    start_time = time.time()
    result = rag_pipeline({"query": question})
    end_time = time.time()
    
    print(f"Question: {question}")
    print(f"Answer: {result['result']}")
    print(f"Time taken: {end_time - start_time:.2f} seconds")
    print("\nSource documents:")
    for i, doc in enumerate(result["source_documents"]):
        print(f"Document {i+1}:")
        print(f"Source: {doc.metadata.get('source', 'Unknown')}")
        print(f"Content: {doc.page_content[:150]}...\n")

In [12]:
ask_question("What is RAG and how does it work?")
ask_question("What is llama.cpp?")
ask_question("How does LocalAI relate to cloud AI services?")

C:\Users\rishi\AppData\Local\Temp\ipykernel_13504\1799130441.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag_pipeline({"query": question})


Question: What is RAG and how does it work?
Answer: RAG is a technique that combines retrieval-based and generation-based approaches for natural language processing tasks. It works by first retrieving relevant information from a knowledge base, then using this information as additional context for language generation. This helps to ground the model's responses in factual information, reducing hallucinations.
Time taken: 61.59 seconds

Source documents:
Document 1:
Source: docs\sample.txt
Content: Retrieval-Augmented Generation (RAG) is a technique that combines retrieval-based and generation-based approaches
    for natural language processing ...

Document 2:
Source: docs\sample.txt
Content: The llama.cpp library is a C/C++ implementation of Meta's LLaMA model, optimized for CPU usage. It allows running LLaMA models
    on consumer hardwar...

Question: What is llama.cpp?
Answer: llama.cpp is a C/C++ implementation of Meta's LLaMA model, optimized for CPU usage. It allows running LLaM

In [15]:
ask_question("What is summary of foreward written by John Crupi in book enterprise integration patterns?")

Question: What is summary of foreward written by John Crupi in book enterprise integration patterns?
Answer: In the forward written by John Crupi, he talks about how the idea for writing this book started during a conversation with Martin Fowler in 2001. He mentions that while working on "Patterns of Enterprise Application Architecture," Martin realized that the book only touched briefly on integration and suggested they write a separate book on message-based integration patterns. Crupi then joined forces with Kyle and Bobby to work on the project, which eventually became "Enterprise Integration Patterns." He highlights the importance of the book's focus on messaging systems and how it will teach readers how to use them effectively in their own applications. Finally, he encourages readers to explore the supporting website for additional information and to provide feedback on the book.
Time taken: 64.58 seconds

Source documents:
Document 1:
Source: docs\enterprise integration patterns.

In [16]:
ask_question("What all wrote foreward in book enterprise integration patterns?")

Question: What all wrote foreward in book enterprise integration patterns?
Answer: The foreword to the book "Enterprise Integration Patterns" was written by:
Martin Fowler, Kyle Brown, Rachel Reinitz, John Crupi, Bobby Bueker, and Gregor Hohpe.
Time taken: 39.72 seconds

Source documents:
Document 1:
Source: docs\enterprise integration patterns.pdf
Content: them that you may not have been aware of. Finally, the pattern names give you a common 
vocabulary to efficiently discuss integration design alternati...

Document 2:
Source: docs\enterprise integration patterns.pdf
Content: immediately following this one in the book. Use the web of interconnected patterns, not the 
linear list of book pages, to guide you through the mater...

Document 3:
Source: docs\enterprise integration patterns.pdf
Content: • When to send a message, what it should contain, and how to use special message 
properties  
• How to route a message to its ultimate destination ev...

